<a href="https://colab.research.google.com/github/MustiCankan/SpiriteAI/blob/main/GenerateImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [16]:
import gradio as gr
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

# # Optional: Use DPM++ 2M Karras scheduler for crisper fine details
# from diffusers import EDMDPMSolverMultistepScheduler
# pipe.scheduler = EDMDPMSolverMultistepScheduler()

def generate_image(prompt,step_size,guidance_scale):
  image = pipe(prompt=prompt, num_inference_steps=step_size, guidance_scale=guidance_scale).images[0]
  return image

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Group():
          gr.Markdown('Spirit Model Creator v1.0.0')
          logo = gr.Image(value="/content/spırıt.png", show_label=False, elem_id="logo-image")
        with gr.Column():
            prompt = gr.Textbox(label = "Prompt")
            with gr.Accordion("Advanced options", open=False):
              step_size = gr.Number(label = "Step Size",value= 50)
              guidance_scale = gr.Number(label = "Guidance Scale",value = 3)
            generate_image_btn = gr.Button(value="Generate Image")
            examples = gr.Examples(examples=["Blonde hair girl", "Bald hair girl"],inputs= [prompt])


        with gr.Column():
          output_image = gr.Image(label="Generated Image")



        # Custom CSS for resizing the logo
        demo.css = """
        #logo-image img {
            width: 200px;
            height: auto;
            display: block;
        }
        """


    generate_image_btn.click(generate_image, inputs=[prompt,step_size,guidance_scale], outputs=output_image)


demo.launch(share=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb16322d42fd2d8451.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
